# JuMP benchmark : Best Linear solver, backend and solver
## *Goddard Rocket Problem*

**Remarques** 
> MOI.AutomaticDifferentiationBackend() is not supported by KNITRO

> With MUMPS, ExaModel is much faster than JuMP ( possible reason : we use `ipopt()` directy while solving an ExaModel while we use `optimize()` with JuMP)

In [1]:
using JuMP
include("../Problems/JuMP/rocket_JMP.jl")
include("./linear_solver_variant.jl")
include("./backend_variant.jl")
include("./solver_variant.jl")
include("./utils.jl")
JuMPModel = rocket_JMP();

set_silent(JuMPModel)
set_optimizer(JuMPModel,Ipopt.Optimizer)
set_optimizer_attribute(JuMPModel,"tol",1e-8)
set_optimizer_attribute(JuMPModel,"constr_viol_tol",1e-6)
set_optimizer_attribute(JuMPModel,"max_iter",1000)

## Linear solver

In [2]:
sl_results,best_ls = linear_solver_variant(JuMPModel);

Using Ipopt Solver :
Solving With MUMPS...
******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

✅

Solving With HSLMA57...✅

Solving With HSLMA27...✅

-------------------------Results of Linear Solvers-------------------------
3×4 DataFrame
 Row │ method   diff_auto_time  total_time   objective_value 
     │ String   Float64         String       Float64         
─────┼───────────────────────────────────────────────────────
   1 │ HSLMA57       0.016      14.45005 ms          1.01283
   2 │ HSLMA27       0.0149999  15.3946 ms           1.01283
   3 │ MUMPS         0.03       27.0203 ms           1.01283


In [3]:
if best_ls == "HSLMA27" 
    println("The best linear solver is HSLMA27")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma27")
elseif best_ls == "HSLMA57"
    println("The best linear solver is HSLMA57")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma57")
elseif best_ls == "MUMPS"
    println("The best linear solver is MUMPS")
    set_attribute(JuMPModel, "linear_solver", "mumps")
end

The best linear solver is HSLMA57


## Backend

In [4]:
be_results, best_backend = backend_variant(JuMPModel);

Solving With : ExaModels...✅

Solving With : JuMPDefault (MathOptInterface.Nonlinear.SparseReverseMode())...✅

Solving With : SymbolicAD (MathOptSymbolicAD.DefaultBackend())...✅

-------------------------Results of Backends-------------------------
3×4 DataFrame
 Row │ method       diff_auto_time  total_time   objective_value 
     │ String       Float64         String       Float64         
─────┼───────────────────────────────────────────────────────────
   1 │ JuMPDefault       0.0209999  16.1498 ms           1.01283
   2 │ SymbolicAD        0.016      16.17595 ms          1.01283
   3 │ ExaModels         0.019      23.7479 ms           1.01283


In [5]:
if best_backend == "SymbolicAD" 
    println("The best backend is SymbolicAD")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MathOptSymbolicAD.DefaultBackend(),)
elseif best_backend == "JuMPDefault"
    println("The best backend is JuMPDefault")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MOI.Nonlinear.SparseReverseMode(),)
elseif best_backend == "ExaModels"
    println("The best linear solver is ExaModels")
    println("ExaModels works only with ipopt and madnlp solvers")
    println("For the sake of the rest of the benchmark, we'll use the 2nd best backend $((be_results[!,1])[2])")
    second_best_backend = (be_results[!,1])[2]
    if second_best_backend == "SymbolicAD" 
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MathOptSymbolicAD.DefaultBackend(),)
    elseif second_best_backend == "JuMPDefault"
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MOI.Nonlinear.SparseReverseMode(),)
    end
end

The best backend is JuMPDefault


## solver 

In [6]:
s_results,best_s = solver_variant(JuMPModel);

Solving With Ipopt...✅

Solving With MadNLP...✅

Solving With KNITRO_SQP...✅

Solving With KNITRO_IPM...✅

-------------------------Results of Solvers-------------------------
4×4 DataFrame
 Row │ method      diff_auto_time  total_time  objective_value 
     │ String      Float64         String      Float64         
─────┼─────────────────────────────────────────────────────────
   1 │ KNITRO_SQP       0.0        407.3 μs            1.01283
   2 │ KNITRO_IPM       0.0        747.0 μs            1.01283
   3 │ Ipopt            0.0380001  30.8041 ms          1.01283
   4 │ MadNLP           0.039      45.2078 ms          1.01283


In [7]:
if best_s == "KNITRO_IPM" 
    println("The best solver is KNITRO_IPM")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 1);
elseif best_s == "Ipopt"
    println("The best solver is Ipopt")
    set_optimizer(JuMPModel,Ipopt.Optimizer);
elseif best_s == "MadNLP"
    println("The best solver is MadNLP")
    set_optimizer(JuMPModel,MadNLP.Optimizer);
elseif best_s == "KNITRO_SQP"
    println("The best solver is KNITRO_SQP")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 4);
end

The best solver is KNITRO_SQP
